In [1]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [2]:
Podsumowanie_paczek_i_Pw("24/46")

OBIĘTOSC PACZEK (Z OWATAMI)
20/01: 132m3
21/01: 162m3
WST: 38m3
Objetość wszystkich paczek: 332m3
PW 24/46 spakowano: 86M3


In [9]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"])
br.to_excel("BRAKI.xlsx")

In [4]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

WOLNE PONIZEJ MIN: 66 POZYCJE
WOLNE PONIZEJ MIN NIE ZAMOWIONE: 46 POZYCJE
SALDO PONIZEJ MIN: 39 POZYCJE
SALDO PONIZEJ MIN NIE ZAMOWIONE: 25 POZYCJE


,OPIS,ZAMOWIONE,CZEKA_NA_SPAKOWANIE,SALDO,MIN,WOLNE_SALDO
0,AVANT [1],0.0,0.0,0.0,2,0.0
8,"AVANT 2,5",0.0,0.0,3.0,5,3.0
13,AVANT 2][,0.0,0.0,3.0,7,2.0
19,AVANT LM][,0.0,0.0,10.0,15,8.0
20,AVANT P104x64,0.0,0.0,9.0,10,7.0
69,DIVA N,0.0,0.0,1.0,2,0.0
82,DUO P82x82,0.0,0.0,0.0,1,0.0
94,"EXTREME 1,5s",0.0,0.0,0.0,8,0.0
97,"EXTREME [1,5s",0.0,0.0,0.0,8,0.0
107,EXTREME P80x105,0.0,0.0,0.0,8,0.0


In [5]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

MAX_obj:                    3263
ZAMOWIONE_obj:              346
CZEKA_NA_SPAKOWANIE_obj:    219
SALDO_obj:                  2186
WOLNE_obj:                  1854
WOLNE_NIE_SPAK_obj:         2073
DO_ZAM_obj:                 849
---
ZAPEŁNIENIE MAGAZYNU
SALDO / MAX: 67.0%
WOLNE / MAX: 56.8%
WOLNE_NIE_SPAK / MAX: 63.5%
ZAPEŁNIENIE MAG PO ZDJECIU 2 PACZEK RAZEM Z ZAMOWIENIAMI: 74.1%


In [6]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp, instrukcja_zamawiania_ciech as izc
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)



In [7]:
arp

OBJ_CIECH       41.86
OBJ_VITA        27.35
OBJ_PIANPOL    487.10
RAZEM          556.31

In [8]:
arp.Tabela_podsumowania_analizy

,GRUPA,MODEL,BRYL_DO_ZAMOWIENIA,WSPL_DO_ZAM,OBJ_CIECH,OBJ_VITA,OBJ_PIANPOL,*WSPL_ZAP_WOLNE,*WSPL_ZAP_ZAM
2,1,STONE,333.0,46.0,0.00,0.00,124.05,58.0,58.0
1,1,ELIXIR,113.0,31.0,0.00,0.00,49.02,45.0,69.0
0,1,AVANT,236.0,27.0,0.00,0.00,59.65,72.0,77.0
3,1,HORIZON,88.0,10.0,0.00,27.35,5.72,60.0,60.0
8,2,WILLOW,119.0,37.0,0.00,0.00,53.43,63.0,63.0
4,2,AMALFI,129.0,23.0,0.00,0.00,46.16,46.0,80.0
9,2,REVERSO,127.0,20.0,0.00,0.00,37.02,62.0,82.0
5,2,DIVA,46.0,17.0,0.00,0.00,13.64,87.0,87.0
6,2,MAXWELL,27.0,16.0,0.00,0.00,9.66,76.0,92.0
7,2,OXYGEN,26.0,12.0,0.00,0.00,7.73,97.0,97.0


In [ ]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"



In [10]:
# Zapis_danych_do_Archiwum(242X
# , analiza, arp._Tabela_podsumowania_analizy, "sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db")